## Exercícios banco de dados - Módulo 6 (Santander Coders 806)

Alunos: 
- Samya de Lara Pinheiro
- Vitor Quintella  

**mai/2022**

### Exercício 1
> **Loja de Computadores** - Criação de banco no pgAdmin e testes de queries para consultas específicas.  

- Crie um novo banco de dados chamado exercicios (se ele já não existir)
- Execute o código abaixo para criação de tabelas que serão usadas
- Responda às perguntas com o comando SQL logo abaixo de cada item (depois de testar no pgAdmin)

``` sql
CREATE TABLE Manufacturers (
	Code INTEGER PRIMARY KEY NOT NULL,
	Name CHAR(50) NOT NULL
);

CREATE TABLE Products (
	Code INTEGER PRIMARY KEY NOT NULL,
	Name CHAR(50) NOT NULL ,
	Price REAL NOT NULL ,
	Manufacturer INTEGER NOT NULL
		CONSTRAINT fk_Manufacturers_Code REFERENCES Manufacturers(Code)
);

INSERT INTO Manufacturers
    (Code,Name)
VALUES
    (1,'Sony'),
    (2,'Creative Labs'),
    (3,'Hewlett-Packard'),
    (4,'Iomega'),
    (5,'Fujitsu'),
    (6,'Winchester');

INSERT INTO
    Products(Code,Name,Price,Manufacturer)
VALUES
    (1,'Hard drive',240,5),
    (2,'Memory',120,6),
    (3,'ZIP drive',150,4),
    (4,'Floppy disk',5,6),
    (5,'Monitor',240,1),
    (6,'DVD drive',180,2),
    (7,'CD drive',90,2),
    (8,'Printer',270,3),
    (9,'Toner cartridge',66,3),
    (10,'DVD burner',180,2);
```

1. Selecione os nomes de todos os produtos da loja.

```sql
select name from products;
```

2. Selecione os nomes e os preços de todos os produtos da loja.

```sql
select name,price from products;
```

3. Selecione o nome dos produtos com preço menor ou igual a 200.

```sql
select name from products where price <= 200;
```

4. Selecione todos os produtos com preço entre $60 e $120.

```sql
select name from products where price between 60 and 120;
```

5. Selecione o nome e o preço em centavos (ou seja, o preço deve ser multiplicado por 100).

```sql
select name,price*100 from products;
```

6. Calcule o preço médio de todos os produtos.

```sql
select avg(price) as media_preco from products;
```

7. Calcule o preço médio de todos os produtos com código de fabricante igual a 2.

```sql
select avg(price) as media_preco from products where manufacturer = 2;
```

8. Calcule o número de produtos com preço maior ou igual a $180.

```sql
select count(*) as num_produtos from products where price >= 180;
```

9. Selecione o nome e o preço de todos os produtos com preço maior ou igual a US$ 180 e classifique primeiro por preço (em ordem decrescente) e depois por nome (em ordem crescente).

```sql
select name,price from products where price >= 180 order by price desc,name;
```

10. Selecione todos os dados dos produtos, incluindo todos os dados do fabricante de cada produto.

```sql
select
    products.*
    ,manufacturers.name as manufacturer_name
from products
inner join manufacturers
on products.manufacturer = manufacturers.code
order by manufacturer, products.name;
```

11.Selecione o nome do produto, preço e nome do fabricante de todos os produtos.

```sql
select
    products.name as product_name
    ,products.price as price
    ,manufacturers.name as manufacturer_name
from products
inner join manufacturers
on products.manufacturer = manufacturers.code
order by manufacturer, products.name;
```

12. Selecione o preço médio dos produtos de cada fabricante, mostrando apenas o código do fabricante.

```sql
select manufacturer, avg(price) from products group by manufacturer;
```

13. Selecione o preço médio dos produtos de cada fabricante, mostrando o nome do fabricante.

```sql
-- opcao 1: Mais limpo
select m.name, avg(p.price) from products p inner join manufacturers m on p.manufacturer = m.code group by m.name;

-- opcao 2: Menos custoso? (testar com explain)
select name,media_preco
from
(select manufacturer, avg(price) as media_preco
from products group by manufacturer )
    as t
inner join manufacturers
    on t.manufacturer = manufacturers.code;
```

14. Selecione os nomes dos fabricantes cujos produtos têm preço médio maior ou igual a 50.


```sql
with cte_media as (
    select m.name, avg(p.price) as media_preco
    from products p
    inner join manufacturers m on p.manufacturer = m.code
    group by m.name
    )
select name from cte_media where media_preco >= 150;
```

15. Selecione o nome e o preço do produto mais barato.

```sql
select name,price from products order by price asc limit 1;

-- opcao 2
with cte_min as (select min(price) as min_price from products)
select name, price from products
inner join cte_min on price = cte_min.min_price;
```

16. Selecione o nome de cada fabricante junto com o nome e o preço de seu produto mais caro.

```sql
-- seleciona considerando a possibilidade de mais de um produto com o maior preço
with cte_max as (select manufacturer,max(price) as max_price from products group by manufacturer)
select manufacturers.name, products.name, price from products
right join  cte_max on price = cte_max.max_price and products.manufacturer = cte_max.manufacturer
inner join manufacturers on manufacturers.code = products.manufacturer
order by manufacturers.name, products.name;
```

17. Selecione o nome de cada fabricante que tenha um preço médio acima de 145 e contenha pelo menos 2 produtos diferentes.

```sql
with cte_media as (
    select m.name, avg(p.price) as media_preco,  count(distinct p.name) as qtd_produtos
    from products p
    inner join manufacturers m on p.manufacturer = m.code
    group by m.name
    )
select name from cte_media where media_preco > 145 and qtd_produtos> 1;
```

18. Adicione um novo produto: Alto-falantes, US 70, fabricante 2.

```sql
insert into products
        (code, name, price, manufacturer)
    values
        ((select MAX(code)+1 from products), 'Alto-falantes', 70, 2);
```

19. Atualize o nome do produto 8 para "Impressora a Laser".

```sql
UPDATE products SET name = 'Impressora a Laser' WHERE code = 8;
```

20. Aplique um desconto de 10% em todos os produtos.

```sql
update products set price = price * (1-0.1);
```

21. Aplique um desconto de 10% em todos os produtos com preço maior ou igual a 120.

```sql
update products set price = price * (1-0.1) where price >= 120;
```

### Exercício 2
> **Gerenciamento de Empregados** - Criação de banco no pgAdmin e testes de queries para consultas específicas.  

![Relacionamento](https://upload.wikimedia.org/wikipedia/commons/thumb/3/36/Employees.png/300px-Employees.png)

- Crie as tabelas e relacionamentos conforme diagrama acima.
- Insira os valores abaixo em suas respectivas tabelas.
- Responda àss perguntas com o comando SQL logo abaixo de cada item (depois de testar no pgAdmin)  

**OPTAMOS PELA CRIAÇÃO DO DATABASE E TABELAS ATRAVÉS DA INTEGRAÇÃO EM PYTHON PARA TREINAR A APLICAÇÃO**

#### Criação da Base de Dados

In [4]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# read pwd file
with open('postgremdp.txt') as f:
    mdp = f.readlines()[0]

# open connection with postgresql
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password=mdp)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# obtain a DB Cursor
cursor = conn.cursor()

# create sql statement to creat database
name_Database = "Employees"
sqlCreateDatabase = "create database "+name_Database+";"

# create a PostgreSQL database
cursor.execute(sqlCreateDatabase)

cursor.close()
conn.close()
print('Base Employees criada!')

Base Employees criada!


#### Criando as Tabelas com respectivos relacionamentos

In [5]:
## Create Table
# create table queries
table_dept = "CREATE TABLE IF NOT EXISTS Departments(Code INTEGER PRIMARY KEY NOT NULL, Name TEXT NOT NULL, Budget MONEY NOT NULL)"
table_employ = "CREATE TABLE IF NOT EXISTS Employees(SSN INTEGER PRIMARY KEY NOT NULL, Name TEXT NOT NULL, LastName TEXT NOT NULL, Department INTEGER NOT NULL, CONSTRAINT FK_Department_Code FOREIGN KEY(Department) REFERENCES Departments(Code))"

# connect
conn = psycopg2.connect(
    host="localhost",
    database="employees",
    user="postgres",
    password=mdp)
# cursor
cur = conn.cursor()

print("Criando Tabelas...")

try:
    cur.execute(table_dept)   
    conn.commit()
    cur.execute(table_employ)   
    conn.commit()
    cur.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Tabelas criadas! Conexão encerrada.')

Criando Tabelas...
Tabelas criadas! Conexão encerrada.


#### Inserindo valores nas tabelas

In [6]:
## Insert rows
# insert function Departments
def insert_depart(_Code, _Name, _Budget):
    
    """ Insere um novo registro na tabela Departments """
    
    sql = f"INSERT INTO Departments (Code, Name, Budget) VALUES({_Code},'{_Name}','{_Budget}') ;"
    conn = None
    
    try:
        ## connect
        conn = psycopg2.connect(
            host="localhost",
            database="employees",
            user="postgres",
            password=mdp)
        ## cursor
        cur = conn.cursor()

        print("Inserindo em Tabela Departments...")
        # execute the INSERT statement
        cur.execute(sql)
        conn.commit()
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return print(_Code)

# insert values
insert_depart(14, 'IT', 65000)
insert_depart(37, 'Accounting', 15000)
insert_depart(59, 'Human Resources', 240000)
insert_depart(77, 'Research', 55000)

Inserindo em Tabela Departments...
14
Inserindo em Tabela Departments...
37
Inserindo em Tabela Departments...
59
Inserindo em Tabela Departments...
77


In [7]:
# read csv for employees
import pandas as pd
employ_df = pd.read_csv('employees.csv')
employ_df

,ssn,name,lastname,department
0,123234877,Michael,Rogers,14
1,152934485,Anand,Manikutty,14
2,222364883,Carol,Smith,37
3,326587417,Joe,Stevens,37
4,332154719,Mary-Anne,Foster,14
5,332569843,George,O'Donnell,77
6,546523478,John,Doe,59
7,631231482,David,Smith,77
8,654873219,Zacary,Efron,59
9,745685214,Eric,Goldsmith,59


In [8]:
## Loop to insert values in table Employees
# connect
conn = psycopg2.connect(
    host="localhost",
    database="employees",
    user="postgres",
    password=mdp)
# cursor
cur = conn.cursor()

# list for inserting
cols = ",".join([str(i) for i in employ_df.columns.tolist()])

# insert DataFrame records one by one.
for i,row in employ_df.iterrows():
    sql = "INSERT INTO Employees (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    print('Inserida linha ' + str(i))
    cur.execute(sql, tuple(row))
    conn.commit()

cur.close()
conn.close()
print('Tabela Employees povoada! Conexão encerrada.')

Inserida linha 0
Inserida linha 1
Inserida linha 2
Inserida linha 3
Inserida linha 4
Inserida linha 5
Inserida linha 6
Inserida linha 7
Inserida linha 8
Inserida linha 9
Inserida linha 10
Inserida linha 11
Tabela Employees povoada! Conexão encerrada.


#### Consultas em SQL

### Exercício 3
> **Titanic** - Utilizando base de dados em CSV para criação de Database SQL e execução de consultas.  

- Utilize o site do Kaggle. Iremos baixar o arquivo CSV do Titanic: https://www.kaggle.com/competitions/titanic/data?select=test.csv
- Abaixo o dicionário de dados, que também está no Kaggle. Para este exercício, vamos manter a descrição em inglês 

```raw
Variable	Definition	    Key
survival	Survival	    0 = No, 1 = Yes
pclass	    Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	        Sex	
Age	        Age in years	
sibsp	    # of siblings / spouses aboard the Titanic	
parch	    # of parents / children aboard the Titanic	
ticket	    Ticket number	
fare	    Passenger fare	
cabin	    Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
```

- O objetivo desta tarefa é:  
    a. Criar quatro tabelas: Survival, pclass, embarked e passageiros. Isso por que para as colunas que possuem uma descrição (Key) é necessário criar uma tabela para cada uma delas.  
    b. Usar o arquivo CSV e carregar os respectivos dados em suas tabelas. A criação das tabelas e a inclusão de registros deverá ser feito pelo python  
    c. Nas tabelas survival, pclass e emarked não deverá ter valores duplicados  
    d. Responder às perguntas abaixo via SQL  

#### Criação da Base de Dados

In [14]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# read pwd file
with open('postgremdp.txt') as f:
    mdp = f.readlines()[0]

# open connection with postgresql
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password=mdp)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# obtain a DB Cursor
cursor = conn.cursor()

# create sql statement to creat database
name_Database = "Titanic"
sqlCreateDatabase = "create database "+name_Database+";"

# create a PostgreSQL database
cursor.execute(sqlCreateDatabase)

cursor.close()
conn.close()
print('Base Titanic criada!')

Base Titanic criada!


#### Criando as Tabelas com respectivos relacionamentos

In [19]:
## Create Table
# create table queries
table_survival = "CREATE TABLE IF NOT EXISTS Survival(survival_id INTEGER NOT NULL PRIMARY KEY, Is_Survivor TEXT NOT NULL)"
table_pclass = "CREATE TABLE IF NOT EXISTS Pclass(pclass_id INTEGER NOT NULL PRIMARY KEY, Ticket_class TEXT NOT NULL)"
table_embarked = "CREATE TABLE IF NOT EXISTS Embarked(embarked_id VARCHAR(1) NOT NULL PRIMARY KEY, Port_of_Embarkation TEXT NOT NULL)"
table_passengers = "CREATE TABLE IF NOT EXISTS Passengers(passenger_id INTEGER NOT NULL PRIMARY KEY, Survivor INTEGER NOT NULL, Pclass INTEGER NOT NULL, Name TEXT NOT NULL, Sex TEXT NOT NULL, Age FLOAT,	SibSp INTEGER NOT NULL,	Parch INTEGER NOT NULL, Ticket TEXT NOT NULL, Fare FLOAT NOT NULL, Cabin TEXT, Embarked VARCHAR(1), FOREIGN KEY (Survivor) REFERENCES Survival(survival_id), FOREIGN KEY (Pclass) REFERENCES Pclass(pclass_id), FOREIGN KEY (Embarked) REFERENCES Embarked(embarked_id))"

# connect
conn = psycopg2.connect(
    host="localhost",
    database="titanic",
    user="postgres",
    password=mdp)
# cursor
cur = conn.cursor()

print("Criando Tabelas...")

try:
    cur.execute(table_survival)   
    conn.commit()
    cur.execute(table_pclass)   
    conn.commit()
    cur.execute(table_embarked)   
    conn.commit()
    cur.execute(table_passengers)   
    conn.commit()
    cur.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Tabelas criadas! Conexão encerrada.')

Criando Tabelas...
Tabelas criadas! Conexão encerrada.


#### Inserindo valores nas tabelas

In [12]:
import pandas as pd
df = pd.read_csv("train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
## Loop to insert values in table Employees
# connect
conn = psycopg2.connect(
    host="localhost",
    database="Titanic",
    user="postgres",
    password=mdp)
# cursor
cur = conn.cursor()

# list for inserting
cols = ",".join([str(i) for i in employ_df.columns.tolist()])

# insert DataFrame records one by one.
for i,row in employ_df.iterrows():
    sql = "INSERT INTO Passengers (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    print('Inserida linha ' + str(i))
    cur.execute(sql, tuple(row))
    conn.commit()

cur.close()
conn.close()
print('Tabela Passengers povoada! Conexão encerrada.')

1 - Quantas pessoas sobreviveram do sexo feminino?

```sql
SELECT count(passenger_id) FROM Passengers WHERE survival_id = 1 AND sex = 'female';
```

2 - Quantas pessoas do sexo masculino não sobreviveram?

```sql
SELECT count(passenger_id) FROM Passengers WHERE survival_id = 0 AND sex = 'male';
```

3 - Agrupar em um único resultado (mas pode ser usado varias queries) quantas pessoas sobreviveram de 0 a 10 anos, sobreviveram de 11 a 20 anos, sobreviveram de 21 a 40 anos e sobreviveram acima de 40 anos. O resultado devera ter duas colunas, o nome do grupo (por ex: Pessoas de 0 a 10 anos) e a contagem